In [37]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import wandb
import os
from torch.utils.data import DataLoader, TensorDataset

In [38]:
os.environ['WANDB_API_KEY'] = 'ee8848e4b81bff3ea50b004f3b28ce2deb74c1b1'

In [39]:
# Generate Noisy XOR Dataset
class NoisyXORGenerator:
    def __init__(self, s=0.1, seed=None):
        self.s = s
        self.seed = seed
        self.cluster_centers = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
        self.cluster_labels = np.array([0, 1, 1, 0])
    
    def generate(self, n_samples):
        if self.seed is not None:
            np.random.seed(self.seed)
        
        samples_per_cluster = n_samples // 4
        remainder = n_samples % 4
        
        X = []
        y = []
        
        for i in range(4):
            n = samples_per_cluster + (1 if i < remainder else 0)
            eta = np.random.randn(n, 2)
            cluster_samples = self.cluster_centers[i] + self.s * eta
            X.append(cluster_samples)
            y.append(np.full(n, self.cluster_labels[i]))
        
        X = np.vstack(X)
        y = np.concatenate(y)
        
        indices = np.random.permutation(n_samples)
        X = X[indices]
        y = y[indices]
        
        return X, y
    
    def generate_train_test(self, n_train=2000, n_test=500):
        X_train, y_train = self.generate(n_train)
        old_seed = self.seed
        if self.seed is not None:
            self.seed += 1
        X_test, y_test = self.generate(n_test)
        self.seed = old_seed
        return X_train, X_test, y_train, y_test

# Generate data
generator = NoisyXORGenerator(s=0.1, seed=42)
X_train, X_test, y_train, y_test = generator.generate_train_test(n_train=2000, n_test=500)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train).unsqueeze(1)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test).unsqueeze(1)

# Create DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [40]:
# Define the Neural Network Model
class DynamicXORModel(nn.Module):
    def __init__(self, input_dim=2, hidden_layers=1, hidden_dim=10, output_dim=1):
        super(DynamicXORModel, self).__init__()
        layers = []
        current_dim = input_dim
        
        # Create hidden layers
        for _ in range(hidden_layers):
            layers.append(nn.Linear(in_features=current_dim, out_features=hidden_dim))
            layers.append(nn.Tanh())
            current_dim = hidden_dim
        
        # Output layer
        layers.append(nn.Linear(in_features=current_dim, out_features=output_dim))
        layers.append(nn.Sigmoid())
        
        self.network = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.network(x)

In [ ]:
# Training Loop with W&B Logging
def train_with_wandb(model, train_loader, val_loader, criterion, optimizer, epochs, config):
    # Initialize W&B
    wandb.init(
        project="noisy-xor-classification",
        config=config,
        name=f"depth{config['hidden_layers']}_width{config['hidden_dim']}"
    )
    
    # Watch the model
    wandb.watch(model, criterion, log="all", log_freq=10)
    
    for epoch in range(epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            predicted = (outputs > 0.5).float()
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
        
        epoch_train_loss = running_loss / len(train_loader.dataset)
        epoch_train_acc = 100 * correct / total
        
        # Validation phase
        model.eval()
        running_val_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for inputs, targets in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                running_val_loss += loss.item() * inputs.size(0)
                
                predicted = (outputs > 0.5).float()
                total += targets.size(0)
                correct += (predicted == targets).sum().item()
        
        epoch_val_loss = running_val_loss / len(val_loader.dataset)
        epoch_val_acc = 100 * correct / total
        
        # Log metrics to W&B
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": epoch_train_loss,
            "train_accuracy": epoch_train_acc,
            "val_loss": epoch_val_loss,
            "val_accuracy": epoch_val_acc,
            "learning_rate": optimizer.param_groups[0]['lr']
        })
    
    wandb.finish()

In [42]:
# Configuration
config = {
    "architecture": "DynamicXORModel",
    "input_dim": 2,
    "hidden_layers": 3,
    "hidden_dim": 10,
    "output_dim": 1,
    "optimizer": "Adam",
    "learning_rate": 0.01,
    "batch_size": 64,
    "epochs": 50,
    "loss_function": "BCELoss",
}

# Initialize model
model = DynamicXORModel(
    input_dim=config["input_dim"],
    hidden_layers=config["hidden_layers"],
    hidden_dim=config["hidden_dim"],
    output_dim=config["output_dim"]
)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])

In [43]:
# Train the model with W&B logging
train_with_wandb(
    model=model,
    train_loader=train_loader,
    val_loader=test_loader,
    criterion=criterion,
    optimizer=optimizer,
    epochs=config["epochs"],
    config=config
)

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_accuracy,▁▆██████████████████████████████████████
train_loss,█▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁███████████████████████████████████████
val_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,50
learning_rate,0.01
train_accuracy,100
train_loss,6e-05
val_accuracy,100


In [44]:
# Hyperparameter sweep configuration
sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'parameters': {
        'hidden_layers': {
            'values': [1, 2, 3, 5]
        },
        'hidden_dim': {
            'values': [5, 10, 20, 30]
        },
        'learning_rate': {
            'distribution': 'log_uniform_values',
            'min': 0.001,
            'max': 0.1
        },
        'batch_size': {
            'values': [32, 64, 128]
        },
        'epochs': {
            'value': 50
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project="noisy-xor-classification")

Create sweep with ID: 30yuxsrz
Sweep URL: https://wandb.ai/victor-vangkilde-university-of-copenhagen/noisy-xor-classification/sweeps/30yuxsrz


In [ ]:
# Training function for sweep
def train_sweep():
    wandb.init()
    
    # Get hyperparameters from wandb
    config = wandb.config
    
    # Recreate data loaders with sweep batch size
    train_loader_sweep = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
    test_loader_sweep = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False)
    
    # Create model with sweep parameters
    model = DynamicXORModel(
        input_dim=2,
        hidden_layers=config.hidden_layers,
        hidden_dim=config.hidden_dim,
        output_dim=1
    )
    
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
    
    # Watch model
    wandb.watch(model, criterion, log="all", log_freq=10)
    
    # Training loop
    for epoch in range(config.epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for inputs, targets in train_loader_sweep:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            predicted = (outputs > 0.5).float()
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
        
        epoch_train_loss = running_loss / len(train_loader_sweep.dataset)
        epoch_train_acc = 100 * correct / total
        
        # Validation phase
        model.eval()
        running_val_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for inputs, targets in test_loader_sweep:
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                running_val_loss += loss.item() * inputs.size(0)
                
                predicted = (outputs > 0.5).float()
                total += targets.size(0)
                correct += (predicted == targets).sum().item()
        
        epoch_val_loss = running_val_loss / len(test_loader_sweep.dataset)
        epoch_val_acc = 100 * correct / total
        
        # Log metrics
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": epoch_train_loss,
            "train_accuracy": epoch_train_acc,
            "val_loss": epoch_val_loss,
            "val_accuracy": epoch_val_acc,
        })
    
    wandb.finish()

In [46]:
# Run the sweep
wandb.agent(sweep_id, train_sweep, count=30)

wandb: Agent Starting Run: 3dn1l28i with config:
wandb: 	batch_size: 32
wandb: 	epochs: 50
wandb: 	hidden_dim: 10
wandb: 	hidden_layers: 1
wandb: 	learning_rate: 0.0257032373853885


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train_accuracy,▁███████████████████████████████████████
train_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,50
train_accuracy,100
train_loss,6e-05
val_accuracy,100
val_loss,5e-05


wandb: Agent Starting Run: jrfye86s with config:
wandb: 	batch_size: 32
wandb: 	epochs: 50
wandb: 	hidden_dim: 5
wandb: 	hidden_layers: 5
wandb: 	learning_rate: 0.0014636630899742852


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_accuracy,▁▁▁▄▄▅▅█████████████████████████████████
train_loss,████▇▆▅▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▁▄▄▅▅██████████████████████████████████
val_loss,███▇▆▆▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,50
train_accuracy,100
train_loss,0.00124
val_accuracy,100
val_loss,0.00121


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: dwfn5rti with config:
wandb: 	batch_size: 128
wandb: 	epochs: 50
wandb: 	hidden_dim: 5
wandb: 	hidden_layers: 5
wandb: 	learning_rate: 0.017353983656265972


...

epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train_accuracy,▁███████████████████████████████████████
train_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,█▁██████████████████████████████████████
val_loss,▂█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,50
train_accuracy,100
train_loss,0.0
val_accuracy,100
val_loss,0.0


wandb: Agent Starting Run: dakavlhg with config:
wandb: 	batch_size: 64
wandb: 	epochs: 50
wandb: 	hidden_dim: 20
wandb: 	hidden_layers: 3
wandb: 	learning_rate: 0.04102007040094736


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train_accuracy,▁███████████████████████████████████████
train_loss,▁█▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,█▁██████████████████████████████████████
val_loss,▃█▂▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,50
train_accuracy,100
train_loss,0.0
val_accuracy,100
val_loss,0.0


wandb: Agent Starting Run: lcvsaxip with config:
wandb: 	batch_size: 32
wandb: 	epochs: 50
wandb: 	hidden_dim: 20
wandb: 	hidden_layers: 3
wandb: 	learning_rate: 0.04349500369469533


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
train_accuracy,▁███████████████████████████████████████
train_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁███████████████████████████████████████
val_loss,█▃▁▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,50
train_accuracy,100
train_loss,0.0
val_accuracy,100
val_loss,0.0
